In [1]:
import sys
!{sys.executable} -m pip install pandas plotly kaleido

In [2]:
import json
import kaleido
import urllib.error
import urllib.request
import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import Image
import warnings
warnings.filterwarnings('ignore')

In [3]:
def line_graph(df, cell_Id):
    df['Discharge_Energy (Wh)'] = pd.to_numeric(
        df['Discharge_Energy (Wh)'], errors='coerce')
    df['Charge_Energy (Wh)'] = pd.to_numeric(
        df['Charge_Energy (Wh)'], errors='coerce')
    df['Discharge Energy Throughput (Wh)'] = df['Discharge_Energy (Wh)'].cumsum(
    )
    df['Charge Energy Throughput (Wh)'] = df['Charge_Energy (Wh)'].cumsum()
    fig = px.line(df, x='Cycle_Index', y=[
        'Discharge Energy Throughput (Wh)', 'Charge Energy Throughput (Wh)'], labels={
        'Cycle_Index': 'Cycle'})
    fig.update_traces(mode='markers+lines', hovertemplate=None)
    filename = 'Energy_throughput_' + cell_Id + '.png'
    pio.write_image(fig, file=filename,
                    format='png', scale=1, width=1200, height=800)



In [4]:

url_list = ['http://batteryarchivemrstutoriallb-436798068.ap-south-1.elb.amazonaws.com:81/download/cells/cycle_data_json/SNL_18650_NMC_15C_0-100_0.5-1C_a',
            'http://batteryarchivemrstutoriallb-436798068.ap-south-1.elb.amazonaws.com:81/download/cells/cycle_data_json/SNL_18650_NMC_25C_0-100_0.5-1C_a',
            'http://batteryarchivemrstutoriallb-436798068.ap-south-1.elb.amazonaws.com:81/download/cells/cycle_data_json/SNL_18650_NMC_35C_0-100_0.5-1C_a']

cell_ids = ['SNL_18650_NMC_15C_0-100_0.5-1C_a',
            'SNL_18650_NMC_25C_0-100_0.5-1C_a',
            'SNL_18650_NMC_35C_0-100_0.5-1C_a']


In [5]:
df = pd.DataFrame()
index = 0
for url in url_list:
    httprequest = urllib.request.Request(
        url, method="GET"
    )
    httprequest.add_header("Cookie", "userId=pghege@gmail.com")
    status = 0
    try:
        with urllib.request.urlopen(httprequest) as httpresponse:
            response = json.loads(httpresponse.read())
            status = 1
    except urllib.error.HTTPError as e:
        print(e)
    df_temp = pd.DataFrame()
    if status:
        count = 1
        for item in response['records']:
            item["cell_id"] = cell_ids[index]
            df_temp = df_temp.append(pd.DataFrame.from_records([item]))
            df = df.append(df_temp)
            if count > 499:
                break
            else:
                count += 1
        line_graph(df_temp, cell_ids[index])
    index += 1
df['Discharge_Energy (Wh)'] = pd.to_numeric(
    df['Discharge_Energy (Wh)'], errors='coerce')
fig = px.histogram(df, x='cell_id', y='Discharge_Energy (Wh)', color='cell_id', labels={
    'cell_id': 'Cell', 'Discharge_Energy (Wh)': 'Discharge Energy Throughput, 500 cyc (Wh)'})
pio.write_image(fig, file='./Discharge_energy_throughput_histogram.png',
                format='png', scale=1, width=1200, height=800)

In [ ]:
Image(filename='Discharge_energy_throughput_histogram.png') 